# Detector Beetle Field Trial - Data Analysis

In [1]:
import pandas as pd
import fnmatch
import os
import gpxpy
import re

## Dataset

Dataset is a CSV text file, **detector_beetles1.csv** with the following fields:

* **beetle_id** Code engraved on beetle's elytrum
* **frequency** frequency (MHz) of radio transmitter glued to beetle's pronotum
* **rel_site** release site; Asan [13.473904, 144.708537]; Yigo [13.531333, 144.872750]
* **Notes** tracking notes
* **lat2** latitude of final observation (decimal degrees)
* **lon2** longitude of final observation (decimal degrees)
* **t2** timestamp of final observation (ChST)
* **extended_track_bearing** direction of travel from the final observation point (direct observation or radio direction finding)
* **end_point_located** 
* **in_tree** TRUE if end point is in a tree; determined visually or by radio direction finding
* **breeding_site** TRUE if otjer CRB present
* **flight_test_date** date on which beetle was flight tested and measured (ChST)
* **Length** lengthe of elytra (mm)
* **Width** width across elytra (mm)
* **Weight** mass (g)

In [2]:
df_endpoints = pd.read_csv('detector_beetles1.csv')
df_endpoints.sort(columns=['frequency'], inplace=True) # Sort by frequency
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 1000)
df_endpoints

,beetle_id,frequency,rel_site,Notes,lat2,lon2,t2,extended_track_bearing,end_point_located,in_tree,breeding_site,flight_test_date,Sex,Length,Width,Weight
0,2968,148.641,Asan,weak signal coming from swamp across from Asan Park; swamp,13.473933,144.708633,2015-08-11 09:18,180.00,False,NaN,NaN,2015-08-10,f,21.17,16.47,2.641
1,2977,148.671,Asan,Found at base of coco tree; just off beach,13.474933,144.708750,2015-08-11,NaN,True,False,NaN,2015-08-10,f,21.83,16.99,3.622
2,2991,148.693,Asan,Lost west over hill,13.472575,144.707141,2015-08-12 19:58:50,270.00,False,NaN,NaN,2015-08-11,m,25.86,20.63,5.774
3,2981,148.703,Asan,Lost west over hill; no track,NaN,NaN,NaN,270.00,False,NaN,NaN,2015-08-10,f,24.94,18.96,5.206
4,2971,148.732,Yigo,top of coconut tree adjust GPS,13.535104,144.873882,2015-08-12 11:00,NaN,True,True,NaN,2015-08-10,m,22.36,17.55,3.087
5,2929,148.752,NaN,lost; no track,NaN,NaN,2015-08-10,NaN,False,NaN,NaN,2015-08-05,f,25.41,20.29,4.220
6,2985,148.764,Asan,out of range; towards road,13.473374,144.708416,2015-08-10 21:01,180.00,False,NaN,NaN,2015-08-10,f,22.80,18.07,4.017
7,2963,148.782,Yigo,under trailer in the dirt; no track,13.529393,144.870394,2015-08-12 12:36,NaN,True,False,NaN,2015-08-10,m,20.33,16.28,3.179
8,2952,148.792,Yigo,beetle in breadfruit tree; 2952?; wooded,13.530583,144.872400,2015-08-10 09:30,NaN,True,True,True,2015-08-05,f,24.07,18.67,4.229
9,2978,148.820,Yigo,faint signal in woods; no track,13.529850,144.872300,2015-08-11 20:58,196.44,False,NaN,NaN,2015-08-10,m,25.22,20.15,5.213


## Map Beetle Tracks Using a Google Earth KML File

In [3]:
import pyproj
import math
import random

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
utm55n=pyproj.Proj("+init=EPSG:32655") # UTM coords, zone 55N, WGS84 datum

def extended_track(longitude, latitude, distance, bearing, bearing_jitter=0):
    """ Returns a track extended from a point for a given distance in a given direction.
    longitude, latitude: starting point in WGS84
    distance: distance to extend track in meters
    angle: angle in compass degrees (east = 90)
    angle_jitter: range of a uniform random number to be added or subracted from angle;
        for eaxmple, if angle=90 and angle_jitter=0.5, track will have an angle between 89.5 and 90.5
        
    Returns latitude and longitude of end point.
    """
    x, y = pyproj.transform(wgs84, utm55n, longitude, latitude)
    if bearing_jitter > 0:
        bearing = random.uniform(bearing-bearing_jitter, bearing+bearing_jitter)
    angle = math.radians(90.0 - bearing) # angle in radians
    x_final, y_final = (x + distance * math.cos(angle), y + distance * math.sin(angle))
    longitude_final, latitude_final = pyproj.transform(utm55n, wgs84, x_final, y_final) # Convert back to lnn/lat
    return longitude_final, latitude_final

#extended_track(144.70901, 13.47140, 500, 90, 0.5)
# (144.71362549035726, 13.471476650158763)

def get_bearing(lat1, lon1, lat2, lon2):
    """ Returns bearing between two geographical coordinates in decimal degrees
    """
    startLat = math.radians(lat1)
    startLong = math.radians(lon1)
    endLat = math.radians(lat2)
    endLong = math.radians(lon2)
    dLong = endLong - startLong
    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
         if dLong > 0.0:
             dLong = -(2.0 * math.pi - dLong)
         else:
             dLong = (2.0 * math.pi + dLong)
    bearing = (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;
    return bearing

#get_bearing(43.682213, -70.450696, 43.682194, -70.450769)
# 250.20613449

In [43]:
import simplekml

kml = simplekml.Kml()

track_style = simplekml.Style()
track_style.linestyle.color = simplekml.Color.yellow
track_style.linestyle.width = 4

extended_track_style = simplekml.Style()
extended_track_style.linestyle.color = simplekml.Color.orange
extended_track_style.linestyle.width = 2

in_tree_point_style = simplekml.Style()
in_tree_point_style.iconstyle.color = simplekml.Color.red

on_ground_point_style = simplekml.Style()
on_ground_point_style.iconstyle.color = simplekml.Color.yellow

extended_track_point_style = simplekml.Style()
extended_track_point_style.iconstyle.color = simplekml.Color.orange
extended_track_style.iconstyle.scale = 0.1

# Tracks
# Extended tracks identified with a new point at the last observed waypoint

for i, row in df_endpoints.iterrows():
    if row.rel_site == 'Asan':
        coord1 = (144.708537, 13.473904)
    elif row.rel_site == 'Yigo':
        coord1 = (144.87275, 13.531333)
    else:
        continue

    if not pd.isnull(row.lon2) and not pd.isnull(row.lat2):
        coord2 = (row.lon2, row.lat2)
    else:
        coord2 = coord1
        
    ls = kml.newlinestring(name=str(row.beetle_id), coords=[coord1, coord2])
    ls.style = track_style
    
    # Points for beetles with extended tracks
    
    if not pd.isnull(row.extended_track_bearing):
        extended_coord = extended_track(coord2[0], coord2[1], 500, row.extended_track_bearing, 5.0)
        ls = kml.newlinestring(coords=[coord2, extended_coord])
        ls.style = extended_track_style
        pnt = kml.newpoint(name=str(row.beetle_id), coords=[coord2])
        pnt.style = extended_track_point_style
        
# Points for beetles tracked to end points       
# Color code icon red: in tree; yellow: on ground; NaN: ignore

for i, row in df_endpoints.iterrows():
    if row.end_point_located==True and not pd.isnull(row.in_tree):
        pnt = kml.newpoint(name=str(row.beetle_id), coords=[(row.lon2, row.lat2)]) 
        if row.in_tree:
            pnt.style = in_tree_point_style
        else:
            pnt.style = on_ground_point_style
        

kml.save('tracks.kml')

In [27]:
df_endpoints[df_endpoints['beetle_id'].isin([2974,2979,2965])]

,beetle_id,frequency,rel_site,Notes,lat2,lon2,t2,extended_track_bearing,end_point_located,in_tree,breeding_site,flight_test_date,Sex,Length,Width,Weight
15,2974,148.963,Yigo,"edge of dense woods, up high; wooded; no track",13.529167,144.872683,2015-08-11 19:31,181.72,False,True,NaN,2015-08-10,m,22.59,17.97,3.562
26,2979,164.232,Asan,Lost at indicated coordinates; went out of range,13.474440,144.708450,2015-08-10,351.03,False,False,NaN,2015-08-10,m,24.31,19.01,4.563
31,2965,164.357,Asan,towards tall grass only up high,13.472615,144.707277,2015-08-13 11:20,180.00,False,True,NaN,2015-08-10,f,22.13,17.13,2.989


## Generate LaTeX table

In [5]:
df_endpoints.to_latex('table.tex')